In [2]:
class PierrePredictor:
    # Gaussian Process Regression
    
    def __init__(self, data):
        # The dimension of the data
        self.n_dim = len(data)
        
    def K(self, X, Z, h):
        # The Covariance Matrix
        d = spatial.distance_matrix(X,Z)
        K = np.exp(-(d**2) / (2*h*h)) 
        return K
    
    def mean_f(self,h,sigma,X,Z,Y):
        # The predictive mean m(f)
        K_X_X = self.K(X,X,h)
        K_test_X = self.K(Z,X,h)
        Id = np.identity(X.shape[0])
        S=np.linalg.inv(K_X_X+sigma*Id)
        m=np.dot(K_test_X,np.dot(S,Y))
        return m

    def cov_f(self,h,sigma,X,Z):
        # The predictive cov(f)
        K_test = self.K(Z,Z,h)
        K_test_X = self.K(Z,X,h)
        K_X_test = self.K(X,Z,h)
        K_X_X = self.K(X,X,h)
        Id = np.identity(X.shape[0])
        S=np.linalg.inv(K_X_X+sigma*Id)
        cov_f = K_test-np.dot(K_test_X,np.dot(S,K_X_test))
        return cov_f
        
    def L(self,h,sigma,X,Z):
        # The Cholesky decomposition of the predictive cov(f)
        cov_f = self.cov_f(h,sigma,X,Z)
        L = np.linalg.cholesky(cov_f + 0.001*np.eye(cov_f.shape[0]))
        return L
    
    def f_sim(self,h,sigma,X,Z,Y):
        # The conditional stochastic simulations of f
        m = self.mean_f(h,sigma,X,Z,Y)
        L = self.L(h,sigma,X,Z)
        mean_f_u = np.zeros(Z.shape[0])
        Id = np.identity(Z.shape[0])
        u = np.random.multivariate_normal(mean_f_u,Id)
        u.shape=Z.shape[0],1
        fsim = m + np.dot(L,u)
        return fsim
    
    def visualization(self,fsim,n_squared_grid,bounding_box):
        
        # bounding_box = List of the latitude & longitude of the bounding box
        # We reshape fsim in a grid of n_squared_grid * n_squared_grid
        
        fsim_reshaped = np.transpose(np.asarray(np.array_split(fsim,n_squared_grid)))
        
        # Exportation of the picture in a .png file
        
        fig = plt.figure(2)

        cmap = mpl.colors.LinearSegmentedColormap.from_list('my_colormap',
                                                   ['darkblue','blue','lightblue','green','lightgreen','yellow'
                                                    ,'orange','red','darkred']
                                                    ,256)

        img = plt.imshow(fsim_reshaped,interpolation='nearest',cmap = cmap,origin='lower')

        fig.savefig("Picture of the simulated rainfall.png",transparent=True,frameon=False,pad_inches=0,bbox_inches='tight')
        
        # Exportation of the picture in a .kml file, readable by Google Earth
        
        kml = simplekml.Kml()
        ground = kml.newgroundoverlay(name='GroundOverlay')
        ground.icon.href = 'Picture of the simulated rainfall.png'
        
        ground.latlonbox.north = bounding_box[1]
        ground.latlonbox.south = bounding_box[0]
        ground.latlonbox.east = bounding_box[2]
        ground.latlonbox.west = bounding_box[3]
        ground.latlonbox.rotation = 0

        ground.color = "aaffffff"
        kml.save("Picture of the simulated rainfall.kml")
            